****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [59]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as etree
tree = etree.parse( './data/mondial_database.xml' )
root = tree.getroot()
root

<Element 'mondial' at 0x000001EDC0F80C78>

In [52]:
for child in root:
    print(child.tag)

country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country
country


### Print 10 countries with lowest infant mortality rates

In [8]:
inf_mort = pd.DataFrame(columns = ["country", "infant_mortality"])

for element in root.iter('country'):
    ctry = element.find('name').text
    
    if element.find('infant_mortality') == None:
        continue
    mort = element.find('infant_mortality').text
    
    inf_mort = inf_mort.append({
        "country": ctry,
        "infant_mortality": mort
    }, ignore_index=True)
    
inf_mort.infant_mortality = pd.to_numeric(inf_mort.infant_mortality)
inf_mort.sort_values('infant_mortality').head(10)

,country,infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


### 10 cities with the largest population

In [58]:
# Create blank list for dataframe
list_for_df = []

# Loop to find city names and provinces
for country in root.iterfind('country'):
    ctry_cities = country.findall('city')
    provinces = country.findall('province')

# Some countries have province populations; this loop combines them
    for province in provinces:
        ctry_cities += province.findall('city')

# Loops for city name and last population entry
    for city in ctry_cities:
        city_name = city.find('name').text
        population = city.find('.//population[last()]')
# Skips population count if node is empty
        if population != None:
            city_pop = int(population.text)
# Gets the year that the population count is from
            recent_year = population.get('year')
        recent_pop = (country.find('name').text,
                      city_name, recent_year, city_pop)
        list_for_df.append(recent_pop)

# Creates dataframe
labels = ['Country', 'City', 'Year', 'Population']
city_df = pd.DataFrame.from_records(list_for_df, columns=labels)
city_df.sort_values('Population', ascending=False).head(10)

,Country,City,Year,Population
1341,China,Shanghai,2010,22315474
771,Turkey,Istanbul,2012,13710512
1527,India,Mumbai,2011,12442373
479,Russia,Moskva,2013,11979529
1340,China,Beijing,2010,11716620
2810,Brazil,São Paulo,2010,11152344
1342,China,Tianjin,2010,11090314
1064,China,Guangzhou,2010,11071424
1582,India,Delhi,2011,11034555
1067,China,Shenzhen,2010,10358381


### 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [69]:
c_pop = {}

for element in root.iterfind('country'):
    country = element.find('name')
    population = element.find('.//population[last()]')
    
    
    if (country) != None:
        if (population) != None:
            c_pop[country.text] = int(population.text)
    else:
        c_pop[country.text] = "NA"
        
cdf = pd.DataFrame.from_dict(c_pop, orient = 'index')
cdf.reset_index(drop=False, inplace=True)
cdf.columns=['Country', 'Population']
cdf.sort_values('Population', ascending=False).head(10)

,Country,Population
55,China,1360720000
67,India,1210854977
120,United States,318857056
88,Indonesia,252124458
176,Brazil,202768562
57,Pakistan,173149306
202,Nigeria,164294516
65,Bangladesh,149772364
23,Russia,143666931
98,Japan,127298000


In [84]:
ethnic_groups = []
ethnic_group_labels = ['Country', 'Ethnicity', 'Percentage']

for country in root.iterfind('country'):
    c = country.findtext('name')
    
    for e in country.iter('ethnicgroup'):
        ethnicity = e.text
        ethnic_pct = e.attrib['percentage']
        ethnic_groups.append([c, ethnicity, ethnic_pct])
        
edf = pd.DataFrame(ethnic_groups, columns = ethnic_group_labels)
edf = edf.merge(cdf, on='Country')
edf.groupby('Ethnicity').sum().sort_values(by='Population', ascending=False).head(10)

,Population
Ethnicity,
Han Chinese,1360720000
Mongol,1213609662
Indo-Aryan,1210854977
Dravidian,1210854977
European,1157295639
African,975352746
Amerindian,588752467
Malay,377500275
Asian,374650120


### name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [90]:
# First create a dictionary with name of country and its car_code and then create a data from from the dictionary

country_codes = {}
for element in root.iterfind('country'):
    country_codes[element.attrib['car_code']] = element.findtext('name')
    
codes_df = pd.DataFrame.from_dict(country_codes, orient='index')
codes_df.reset_index(drop=False, inplace=True)
codes_df.columns = ['Country Code', 'Country']
codes_df.head()

,Country Code,Country
0,AL,Albania
1,GR,Greece
2,MK,Macedonia
3,SRB,Serbia
4,MNE,Montenegro


### Name and country with longest river

In [97]:
river_list = [] # create empty list

for river in root.iterfind('river'):
    river_name = river.findtext('name')
    river_length_el = river.find('length')
    if river_length_el != None:
        river_length = river_length_el.text
        
        for river_code in river.attrib['country'].split():
            river_ctry_code = country_codes[river_code]
    river_list.append([river_ctry_code, river_code, river_name, float(river_length)])

In [106]:
# create data frame from list
rdf_labels = ['Country', 'Country Code', 'River Name', 'River Length']
rdf = pd.DataFrame.from_records(river_list, columns=rdf_labels)
rdf.sort_values('River Length', ascending=False).head(1)

,Country,Country Code,River Name,River Length
174,Peru,PE,Amazonas,6448.0


### Name and country with largest lake

In [104]:
lake_list = [] # create empty list

for lakes in root.iterfind('lake'):
    lake_name = lakes.findtext('name')
    lake_area_el = lakes.find('area')
    if lake_area_el != None:
        lake_area = float(lake_area_el.text)
        for lake_code in lakes.attrib['country'].split():
            lake_ctry_code = country_codes[lake_code]
    lake_list.append([lake_ctry_code, lake_name, lake_area])

In [107]:
# create data frame from list
ldf_labels = ['Country Code', 'Lake Name', 'Lake Area']
ldf = pd.DataFrame.from_records(lake_list, columns=ldf_labels)
ldf.sort_values('Lake Area', ascending=False).head(1)

,Country Code,Lake Name,Lake Area
54,Turkmenistan,Caspian Sea,386400.0


### Name of airport at highest elevation

In [115]:
airport_list = [] # create empty list

for airports in root.iterfind('airport'):
    airport_name = airports.findtext('name')
    airport_elev_el = airports.find('elevation')
    if airport_elev_el != None:
        airport_elev = airport_elev_el.text
        for airport_code in airports.attrib['country'].split():
            airport_country_code = country_codes[airport_code]
    airport_list.append([airport_country_code, airport_code, airport_name, airport_elev])

In [120]:
# create data frame from list
adf_labels = ['Country', 'Country Code', 'Airport Name', 'Elevation']
adf = pd.DataFrame.from_records(airport_list, columns=adf_labels)
adf['Elevation'] = pd.to_numeric(adf.Elevation)
adf.sort_values('Elevation', ascending=False).head(1)

,Country,Country Code,Airport Name,Elevation
80,Bolivia,BOL,El Alto Intl,4063.0
